<a href="https://colab.research.google.com/github/StavroK/MtySaturdayAI2020/blob/master/Fakenews_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
%cd /content/FakeNews/baseline/fnc-1

/content/FakeNews/baseline/fnc-1


In [78]:
! git clone https://github.com/FakeNewsChallenge/fnc-1.git

fatal: destination path 'fnc-1' already exists and is not an empty directory.


In [0]:
import shutil
shutil.rmtree('/content/PNC1')

**Data Set Analysis**

In this step we will check for a balanced data so we can train the machine learning models without oversampling or undersampling classes. If we see some bias towards one class we need to apply techniques to manage this issue with our data set.

First step is to check if we have all Python packages we need be using to visualize the characteristics of the dataset to check for class balance, and install those missing if any. We will need to join files containing labeling of examples to content of articles for our purpose.


We want to check for: 1. Number of examples in each class, 2. Total Percentage of examples in each class, 3. Distribution of the lenght of each example as we will use its word content to train for classification.


In [79]:
# This command will show you the "Location" of your package
# I am using numpy as is the most common package 
! pip show numpy

Name: numpy
Version: 1.18.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: yellowbrick, xgboost, xarray, wordcloud, umap-learn, torchvision, torchtext, torch, tifffile, thinc, Theano, tensorflow, tensorflow-probability, tensorflow-hub, tensorflow-datasets, tensorboard, tables, statsmodels, spacy, sklearn-pandas, seaborn, scs, scipy, scikit-learn, resampy, PyWavelets, python-louvain, pystan, pysndfile, pymc3, pyemd, pyarrow, plotnine, patsy, pandas, osqp, opt-einsum, opencv-python, opencv-contrib-python, numexpr, numba, np-utils, nibabel, moviepy, mlxtend, mizani, missingno, matplotlib, matplotlib-venn, lucid, lightgbm, librosa, knnimpute, Keras, Keras-Preprocessing, Keras-Applications, kapre, jpeg4py, jaxlib, jax, imgaug, imbalanced-learn, imageio, imagecodecs, hyperopt, h5py, gym, gensim,

In [0]:
# use this other command to list all files under "Location"
# we are checking for the following packages
# altair - enables to build histogram for class representation in examples
# matplotlib - extension of NumPy enables you to embed plots into applications
# pandas - used to clean, transform, manipulate and analize data
# pickle - used to convert a python object into a character stream
# pip - used to manage installation and updates of python packages
# seaborn - based on matplotlib is used to vizualize example lenght by category 
# warnings - used to hide warinings coming from seaborn package
! ls /usr/local/lib/python3.6/dist-packages

In [81]:
# If you are missign a package use this command to install
# remember to be in your distribution packages folder used by colab 
# before you run the instalation command

%cd /usr/local/lib/python3.6/dist-packages

# use these commands to install and check installation
# ! pip install nameofpackage
# ! pip show nameofpackage

! pip install warn
! pip show warn

/usr/local/lib/python3.6/dist-packages
Name: warn
Version: 0.1.0
Summary: A more flexible warning module. 
Home-page: https://github.com/Carreau/warn
Author: Matthias Bussonnier
Author-email: bussonniermatthias@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: 


In [98]:
%cd /usr/local/lib/python3.6/dist-packages
! pip install git+https://github.com/altair-viz/altair

/usr/local/lib/python3.6/dist-packages
  Cloning https://github.com/altair-viz/altair to /tmp/pip-req-build-uc7jrk_x
  Running command git clone -q https://github.com/altair-viz/altair /tmp/pip-req-build-uc7jrk_x
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for altair: filename=altair-4.2.0.dev0-cp36-none-any.whl size=729978 sha256=a31d27f4a312a9c12d43ff33b050698cc49f0d9c3755bfe9a017e4f42cf46085
  Stored in directory: /tmp/pip-ephem-wheel-cache-47z5x4k8/wheels/bc/c8/58/34fe9904ff5a6d2e99a590d4b1f67961ff1145fc21df7766ff
Successfully built altair
  Found existing installation: altair 4.1.0
    Uninstalling altair-4.1.0:
      Successfully uninstalled altair-4.1.0


Now that we have secured environment requirements for python packages that will be declared in the code, next we will import them into the program

In [82]:
%cd /usr/local/lib/python3.6/dist-packages
import altair as alt
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import seaborn as sns
import warnings

/usr/local/lib/python3.6/dist-packages


We want to check for: 1. Number of examples in each class, 2. Total Percentage of examples in each class, 3. Distribution of the lenght of each example as we will use its word content to train for classification.

Data is already included in /content/FakeNews folder. Data is already separated by example content and labeled files, and also in train, test and demo files to keep process clean.


In [83]:
%cd /content/FakeNews/baseline/fnc-1/fnc-1
! ls


/content/FakeNews/baseline/fnc-1/fnc-1
competition_test_bodies.csv		test_bodies.csv
competition_test_stances.csv		test_stances_unlabeled.csv
competition_test_stances_unlabeled.csv	train_bodies.csv
README.md				train_stances.csv
scorer.py				train_stances.random.csv


In [0]:
import pandas as pd
train_df=[]
trainbodies_df = []
trainstances_df = []
testbodies_df = []
teststances_df = []
comptestbodies_df = []
comptestbodies_df = []

trainbodies_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/train_bodies.csv',
                             keep_default_na=False, na_values=[""])
trainstances_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/train_stances.random.csv',
                             keep_default_na=False, na_values=[""])
testbodies_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/test_bodies.csv',
                             keep_default_na=False, na_values=[""])
teststances_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/test_stances_unlabeled.csv',
                             keep_default_na=False, na_values=[""])
comptestbodies_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/competition_test_bodies.csv',
                             keep_default_na=False, na_values=[""])
compteststances_df = pd.read_csv('/content/FakeNews/baseline/fnc-1/fnc-1/competition_test_stances.csv',
                             keep_default_na=False, na_values=[""])

In [85]:
print("Total stances: " + str(len(trainbodies_df)))
print("Total bodies: " + str(len(trainstances_df)))

Total stances: 1683
Total bodies: 49972


In [86]:
trainbodies_df.columns


Index(['Body ID', 'articleBody'], dtype='object')

In [87]:
trainbodies_df.sort_values(by='Body ID')
trainbodies_df.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [88]:
trainstances_df.columns

Index(['Headline', 'Body ID', 'Stance'], dtype='object')

In [89]:
trainstances_df.sort_values(by='Body ID')
trainstances_df.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,discuss
1,Hundreds of Palestinians flee floods in Gaza a...,158,discuss
2,"Christian Bale passes on role of Steve Jobs, a...",137,disagree
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,disagree
4,Spider burrowed through tourist's stomach and ...,1923,agree


In [90]:
train_df = pd.merge(trainbodies_df,trainstances_df, on="Body ID")
print("Total training examples: " + str(len(train_df)))

Total training examples: 49972


In [91]:
train_df.columns

Index(['Body ID', 'articleBody', 'Headline', 'Stance'], dtype='object')

In [92]:
train_df.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",discuss
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,disagree
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,agree
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,agree


In [93]:
train_df.tail()

,Body ID,articleBody,Headline,Stance
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",disagree
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",unrelated
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",discuss
49970,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,disagree
49971,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,agree


In [96]:
import seaborn as sns
sns.set_style("whitegrid")
import altair as alt
alt.renderers.enable("notebook")

import warnings
warnings.filterwarnings("ignore")

ars = alt.Chart(train_df).mark_bar(size=50).encode(
    x=alt.X("Category"),
    y=alt.Y("count():Q", axis=alt.Axis(title='Number of articles')),
    tooltip=[alt.Tooltip('count()', title='Number of articles'), 'Category'],
    color='Category'

)

text = bars.mark_text(
    align='center',
    baseline='bottom',
).encode(
    text='count()'
)

(bars + text).interactive().properties(
    height=300, 
    width=700,
    title = "Number of articles in each category",
)

ValueError: ignored

In [0]:
train_df['id'] = 1
train_df2 = pd.DataFrame(df.groupby('Stance').count()['id']).reset_index()

bars = alt.Chart(train_df2).mark_bar(size=50).encode(
    x=alt.X('Category'),
    y=alt.Y('PercentOfTotal:Q', axis=alt.Axis(format='.0%', title='% of Articles')),
    color='Category'
).transform_window(
    TotalArticles='sum(id)',
    frame=[None, None]
).transform_calculate(
    PercentOfTotal="datum.id / datum.TotalArticles"
)

text = bars.mark_text(
    align='center',
    baseline='bottom',
    #dx=5  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('PercentOfTotal:Q', format='.1%')
)

(bars + text).interactive().properties(
    height=300, 
    width=700,
    title = "% of articles in each category",
)